# Stacked Machine Learning

In [1]:
## IMPORTANT !

# In the first order need to set the number of CPU 
# for calculation before launching (depends on computer's number of cores)
n_jobs= 10

### Load libraries

In [2]:
#libraries
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import pearsonr
import os
import sys
import shutil

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import LeavePGroupsOut
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, datetime
import warnings

import matplotlib.axis as axis

from sklearn.utils import resample
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
import scipy.stats as st

from nilearn import image as nli
from nilearn import plotting
from PIL import Image

from matplotlib.offsetbox import OffsetImage, AnnotationBbox

### Load functions

In [3]:
def control_2(z, control, index):    #age+gender == 2
    #z should be a series
    #control is a feature table
    #index for indexing
    
    #shrink data to local train index
    y = z.reindex(index = index)
    X = control.reindex(index = index)

    #drop Nan in target and clean this subj from features
    y = y.dropna()
    X = X.loc[y.index,:]
    ind_y = np.array(y.index)
    
    #Centralize target by y_i-y_mean
    y= pd.DataFrame([i-y.mean() for i in y], index=y.index)    
    #y_real = y
    
    #reshaping data
    X = X.values
    y = y.values.reshape(-1, 1).ravel()
    
    #fill Nan in X
    X = SimpleImputer(strategy='mean').fit_transform(X)
    
    #Standartize X
    X = StandardScaler().fit_transform(X)
    
    #Fit to the training set
    y_pred = LinearRegression().fit(X, y).predict(X)
    
    y_res = y - y_pred
    
    return y_res, ind_y

In [4]:
def control_mov_feature(z, control, mov, index): #age+gender+each specific task movement == 3
    #z should be a table of features
    #mov should be a series with movements for a specific modality
    
    #shrink data to local train index
    z = z.reindex(index = index)
    control = control.reindex(index = index)
    mov = mov.reindex(index = index) 
    ind = z.index
    #concal control with mov
    cont = control
    cont['mov'] = mov
    
    #loop
    dct = {}
    col_name = z.columns
    for col in col_name:
        y = z[col]
        X = cont
        
        #Centralize target by y_i-y_mean
        y= pd.DataFrame([i-y.mean() for i in y], index=y.index) 
        
        #reshaping data
        X = X.values
        y = y.values.reshape(-1, 1).ravel()

        #fill Nan in X
        X = SimpleImputer(strategy='mean').fit_transform(X)

        #Standartize X
        X = StandardScaler().fit_transform(X)

        #Fit to the training set
        y_pred = LinearRegression().fit(X, y).predict(X)

        y_res = y - y_pred
        
        dct[col] = y_res
    
    df_t = pd.DataFrame(dct, index = ind)
    
    return df_t

In [5]:
def elnet(X, y):

    #drop Nan in target and clean this subj from features
    y = y.dropna()
    X = X.loc[y.index,:]
    ind_y = np.array(y.index)
      
    y_real=y
    
    #reshaping data
    X = X.values
    y = y.values.reshape(-1, 1).ravel()
    
    #fill Nan in X
    X = SimpleImputer(strategy='mean').fit_transform(X)
    
    #Standartize X
    X = StandardScaler().fit_transform(X)
    
    # Setup the pipeline steps:
    steps = [('elasticnet', ElasticNet(random_state=42))]

    # Create the pipeline: pipeline 
    pipeline = Pipeline(steps)

    # Specify the hyperparameter space
    parameters = {'elasticnet__alpha': np.logspace(-6, 4, 500),
                  'elasticnet__l1_ratio':np.linspace(0,1,100)}

    # Create the GridSearchCV object:
    gm_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=n_jobs)
    
    # Fit to the training set
    gm_cv.fit(X, y)
    
    #predict new y
    y_pred = gm_cv.predict(X)

    # Compute and print the metrics
    acc = gm_cv.best_score_
    bpar = gm_cv.best_params_
    model = gm_cv.best_estimator_
    mse = mean_squared_error(y_real, y_pred)
    mae = mean_absolute_error(y_real, y_pred)
    corr, _ = pearsonr(np.array(y_real.values.reshape(-1, 1).ravel(), dtype=float), np.array(y_pred, dtype=float))
            
    return bpar['elasticnet__alpha'], bpar['elasticnet__l1_ratio'], acc, mse, corr, model, y_pred, mae

In [6]:
def reaply_ElNet(X, y, model):
    # param should be pd.Series with indexes from model
    
    #drop Nan in target and clean this subj from features
    y = y.dropna()
    X = X.reindex(index =y.index)
    ind_y = np.array(y.index)  # indexes as separate variable 
    
    y_real = y

    #reshaping data
    X = X.values
    y = y.values.reshape(-1, 1).ravel()
    
    #fill Nan in X
    X = SimpleImputer(strategy='mean').fit_transform(X)
    
    #Standartize X
    X = StandardScaler().fit_transform(X)
    
    #predict new y
    y_pred = model.predict(X)
    
    # Compute and print the metrics
    bacc = model.score(X, y)
    mse = mean_squared_error(y_real, y_pred)
    mae = mean_absolute_error(y_real, y_pred) 
    corr, _ = pearsonr(np.array(y_real.values.reshape(-1, 1).ravel(), dtype=float), np.array(y_pred, dtype=float))
    
    return y_pred, y_real, ind_y, bacc, mse, corr, mae

### Path to the tables folder

In [7]:
path='/media/DataD800/Alina/retest_set/MLtables/'
path_s1200 = '/media/DataD800/Alina/MLtables/'

### Load tables

In [8]:
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#demography
demo_retest_test2 = pd.read_csv(path+'demographics_table.csv', index_col=0)

#targets table
targ_retest_test2 = pd.read_csv(path+'cognition_table.csv', index_col=0)

#features tables as dictionary
features_retest_test2 = {
    'emo':pd.read_csv(path+'emo_table.csv', index_col=0),
    'gam':pd.read_csv(path+'gam_table.csv', index_col=0),
    'lan':pd.read_csv(path+'lan_table.csv', index_col=0),
    'mot':pd.read_csv(path+'mot_table.csv', index_col=0),
    'rel':pd.read_csv(path+'rel_table.csv', index_col=0),
    'soc':pd.read_csv(path+'soc_table.csv', index_col=0),
    'wm':pd.read_csv(path+'wm_table.csv', index_col=0),
    'cort':pd.read_csv(path+'cort_table.csv', index_col=0),
    'subc':pd.read_csv(path+'subc_table.csv', index_col=0),
    'surf':pd.read_csv(path+'surf_table.csv', index_col=0),
    'rest':pd.read_csv(path+'rest_table_featfiltered.csv', index_col=0),
    'VolBrain':pd.read_csv(path+'VolBrain_table.csv', index_col=0)
}


#table with movements (mean relative displacement Movement_RelativeRMS_mean.txt)
movements_retest_test2 = pd.read_csv(path+'movement_table.csv', index_col=0)

#create tables with 2 controling parameters: gender and age
age_coded_retest_test2 = pd.Series(LabelEncoder().fit_transform(demo_retest_test2.loc[:,['Gender']]), index=demo_retest_test2.index, name='Gender')
control_retest_test2 = pd.concat([age_coded_retest_test2, demo_retest_test2.loc[:, ['Age_in_Yrs']]], axis=1)

In [9]:
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#demography
demo_main = pd.read_csv(path_s1200+'demographics_table.csv', index_col=0)

#targets table
targ_main = pd.read_csv(path_s1200+'cognition_table.csv', index_col=0)

#features tables as dictionary
features_main = {
    'emo':pd.read_csv(path_s1200+'emo_table.csv', index_col=0),
    'gam':pd.read_csv(path_s1200+'gam_table.csv', index_col=0),
    'lan':pd.read_csv(path_s1200+'lan_table.csv', index_col=0),
    'mot':pd.read_csv(path_s1200+'mot_table.csv', index_col=0),
    'rel':pd.read_csv(path_s1200+'rel_table.csv', index_col=0),
    'soc':pd.read_csv(path_s1200+'soc_table.csv', index_col=0),
    'wm':pd.read_csv(path_s1200+'wm_table.csv', index_col=0),
    'cort':pd.read_csv(path_s1200+'cort_table.csv', index_col=0),
    'subc':pd.read_csv(path_s1200+'subc_table.csv', index_col=0),
    'surf':pd.read_csv(path_s1200+'surf_table.csv', index_col=0),
    'rest':pd.read_csv(path_s1200+'rest_table_featfiltered.csv', index_col=0),
    'VolBrain':pd.read_csv(path_s1200+'VolBrain_table.csv', index_col=0)
}


#table with movements (mean relative displacement Movement_RelativeRMS_mean.txt)
movements_main = pd.read_csv(path_s1200+'movement_table.csv', index_col=0)

#create tables with 2 controling parameters: gender and age
age_coded_main = pd.Series(LabelEncoder().fit_transform(demo_main.loc[:,['Gender']]), index=demo_main.index, name='Gender')
control_main = pd.concat([age_coded_main, demo_main.loc[:, ['Age_in_Yrs']]], axis=1)

In [10]:
#spliting main ito train and retest_test1

demo_retest_test1 = demo_main.loc[demo_retest_test2.index,:]
targ_retest_test1 = targ_main.loc[demo_retest_test2.index,:]
movements_retest_test1 = movements_main.loc[demo_retest_test2.index,:]
control_retest_test1 = control_main.loc[demo_retest_test2.index,:]
features_retest_test1 = {}
for key in features_main.keys():
    features_retest_test1[key] = features_main[key].loc[demo_retest_test2.index,:]


demo_train = demo_main.drop(demo_retest_test2.index, axis=0)
targ_train = targ_main.drop(demo_retest_test2.index, axis=0)
movements_train = movements_main.drop(demo_retest_test2.index, axis=0)
control_train = control_main.drop(demo_retest_test2.index, axis=0)
features_train = {}
for key in features_main.keys():
    features_train[key] = features_main[key].drop(demo_retest_test2.index, axis=0)



##### Leave-P-groups out based on 8-Fold CV

In [13]:
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


#for col in targ.columns:
col = 'CogTotalComp_Unadj'  
y_train = targ_train[col]
y_retest_test1 = targ_retest_test1[col]
y_retest_test2 = targ_retest_test2[col]


print(y_train.name)

#os.mkdir(path+'output_'+str(y_train.name))
path_out = str(path+'output_'+str(y_train.name))


#Split to local indexes for main train
index_train, index_test = train_test_split(demo_train.index, test_size=0.4, random_state=42)

#Local indices
index_train = np.array(sorted(index_train), dtype='int') #for training modalities models
index_test = np.array(sorted(index_test), dtype='int') #for testing modalities and training second level

index_retest_test = np.array(sorted(demo_retest_test2.index), dtype='int')

print(' ')
print('started to calculate ML')
print(datetime.now())
print(' ')


### 1st level ################################################################################

#### Calculations of single ML models on index_train #################################### 

print('start 1st level ', datetime.now())

#control for age+gen and age+gen+mov with sorting to index_train

#control y (target) for age+gen
p1, p2 = control_2(y_train, control_train, index_train) #where p1 = y_res (residuals), p2 = ind_y (index)
y_res1 = pd.Series(p1, index = p2)

#control modalities
features_res1 = {}
for key in features_train.keys():

    #controlling tasks for 3 parameter (age+gen+mov)
    if key in ['emo', 'gam', 'lan', 'mot', 'rel', 'soc', 'wm', 'rest']:
        features_res1[key] = control_mov_feature(features_train[key], control_train, movements_train[key], index_train)

    #controlling the remaining for 2 parameters (age+gen)
    if key in ['cort', 'surf', 'subc', 'VolBrain']:
        d = {}
        for col in features_train[key].columns:
            p1,p2 = control_2(features_train[key][col], control_train, index_train)
            d[col] = p1
        df= pd.DataFrame(d, index = p2)
        features_res1[key] = df


#Launch ElasticNet for all task(modalities) on index_train (1st level)

dict_tasks={}
dict_elnet_model={}
dict_ypred1={}

for key in list(features_res1.keys()):

    print('start ', str(key), datetime.now())   #print start time of calculations

    bpar1, bpar2, acc, mse, corr, model, y_pred1, mae = elnet(features_res1[key], y_res1) #ML
    dict_tasks[key] = acc, mse, mae, corr, bpar1, bpar2 
    dict_elnet_model[key] = model
    dict_ypred1[key] = y_pred1
df_tasks = pd.DataFrame(dict_tasks, index=['best score r2', 'mse', 'mae','corr', 'best alpha', 'best l1_ratio'])
df_y_pred1 = pd.DataFrame(dict_ypred1, index=y_res1.index)


###Save outputs from this step (models and all mod. perf.)

#model
for key in dict_elnet_model.keys():
    joblib.dump(dict_elnet_model[key], (path_out+'/'+str(key)+'_elnet_model.sav'))

#model performance
df_tasks.to_csv(path_out+'/1level_train_perf_elnet.csv')

#list of first level targets (observed and predicted)
y_res1.to_csv(path_out+'/1level_train_y_real_residual_index.csv', header=None)
df_y_pred1.to_csv(path_out+'/1level_train_y_pred_singleML.csv')


### 2st level ################################################################################
print(' ')
print('start 2nd level ', datetime.now())

#### L2 Testing single ML models on index_test #############################################

print('Checking single ML on test1 data ', datetime.now())

#control for age+gen and age+gen+mov with sorting to index_test

#control y (target) for age+gen
p1, p2 = control_2(y_train, control_train, index_test)
y_res2 = pd.Series(p1, index = p2)

#control modalities
features_res2 = {}
for key in features_train.keys():

    #controlling tasks for 3 parameter (age+gen+mov)
    if key in ['emo', 'gam', 'lan', 'mot', 'rel', 'soc', 'wm', 'rest']:
        features_res2[key] = control_mov_feature(features_train[key], control_train, movements_train[key], index_test)

    #controlling the remaining for 2 parameters (age+gen)
    if key in ['cort', 'surf', 'subc', 'VolBrain']:
        d = {}
        for col in features_train[key].columns:
            p1,p2 = control_2(features_train[key][col], control_train, index_test)
            d[col] = p1
        df= pd.DataFrame(d, index = p2)
        features_res2[key] = df        


#apply trained single models ElasticNet to new data , index_test

dict_y_pred2={}
dict_y_pred2_per={}
for key in list(features_res2.keys()):
    y_pred, y_real, ind_y, bacc, mse, corr, mae = reaply_ElNet(features_res2[key], y_res2, dict_elnet_model[key]) #ML
    dict_y_pred2[key] = y_pred
    dict_y_pred2_per[key] = bacc, mse, mae, corr

df_y_pred2 = pd.DataFrame(dict_y_pred2, index=ind_y)
df_y_pred2_per = pd.DataFrame(dict_y_pred2_per, index=['best score r2', 'mse', 'mae','corr'])


###Save outputs from this step (models and all mod. perf.)

#model performance
df_y_pred2_per.to_csv(path_out+'/2level_test1_perf_elnet.csv')

#list of first level targets (observed and predicted)
y_res2.to_csv(path_out+'/2level_test1_y_real_residual_index.csv', header=None)
df_y_pred2.to_csv(path_out+'/2level_test1_y_pred_singleML.csv')   



#### L2 Calculating stacked ML models on index_test #############################################

print('Calculating stacked ML on test1 data ', datetime.now())    


#identifying sets for several stacked models
set2 = ['emo', 'gam', 'lan', 'mot', 'rel', 'soc', 'wm']
set3 = ['cort', 'subc', 'surf', 'rest', 'VolBrain']
set1 = list(df_y_pred2.columns) #all existed modalities

#for presetet sets
dict_st_perf1={}
dict_st_models={}
dict_st_ypred1={}

s=1
for set_n in [set1, set2, set3]:
    print('set '+str(s), datetime.now())

    st_features = df_y_pred2.loc[:,set_n]

    bpar1, bpar2, acc, mse, corr, model, y_pred3, mae = elnet(st_features, y_res2) #ML

    dict_st_perf1['set'+str(s)] = acc, mse, mae, corr, bpar1, bpar2 
    dict_st_models['set'+str(s)] = model
    dict_st_ypred1['set'+str(s)] = y_pred3
    s+=1

df_st_perf1 = pd.DataFrame(dict_st_perf1, index=['best score r2', 'mse', 'mae','corr', 'best alpha', 'best l1_ratio'])
df_st_ypred1 = pd.DataFrame(dict_st_ypred1, index=y_res2.index)        

###Save outputs from this step (models and all mod. perf.)

#models
for key in dict_st_models.keys():
    joblib.dump(dict_st_models[key], (path_out+'/'+str(key)+'_stacked_model.sav'))

#performance and prediction
df_st_perf1.to_csv(path_out+'/2level_test1_perf_stacked.csv')
df_st_ypred1.to_csv(path_out+'/2level_test1_y_pred_stacked.csv')



### 3rd level ################################################################################
print(' ')
print('start 3rd level , retest1', datetime.now())


#### L3 Testing single ML models on retest1 #############################################

print('Checking single ML on retest1 data ', datetime.now())

#control for age+gen and age+gen+mov with sorting to index_retest_test

#control y (target) for age+gen
p1, p2 = control_2(y_retest_test1, control_retest_test1, index_retest_test)
y_res3 = pd.Series(p1, index = p2)

#control modalities
features_res3 = {}
for key in features_retest_test1.keys():

    #controlling tasks for 3 parameter (age+gen+mov)
    if key in ['emo', 'gam', 'lan', 'mot', 'rel', 'soc', 'wm', 'rest']:
        features_res3[key] = control_mov_feature(features_retest_test1[key], control_retest_test1, movements_retest_test1[key], index_retest_test)

    #controlling the remaining for 2 parameters (age+gen)
    if key in ['cort', 'surf', 'subc', 'VolBrain']:
        d = {}
        for col in features_retest_test1[key].columns:
            p1,p2 = control_2(features_retest_test1[key][col], control_retest_test1, index_retest_test)
            d[col] = p1
        df= pd.DataFrame(d, index = p2)
        features_res3[key] = df        


#apply trained single models ElasticNet to new data , test_index

dict_y_pred3={}
dict_y_pred3_per={}
for key in list(features_res3.keys()):
    y_pred, y_real, ind_y, bacc, mse, corr, mae = reaply_ElNet(features_res3[key], y_res3, dict_elnet_model[key]) #ML
    dict_y_pred3[key] = y_pred
    dict_y_pred3_per[key] = bacc, mse, mae, corr

df_y_pred3 = pd.DataFrame(dict_y_pred3, index=ind_y)
df_y_pred3_per = pd.DataFrame(dict_y_pred3_per, index=['best score r2', 'mse', 'mae','corr'])


###Save outputs from this step (models and all mod. perf.)

#model performance
df_y_pred3_per.to_csv(path_out+'/3level_retest1_perf_elnet.csv')

#list of first level targets (observed and predicted)
y_res3.to_csv(path_out+'/3level_retest1_y_real_residual_index.csv', header=None)
df_y_pred3.to_csv(path_out+'/3level_retest1_y_pred_singleML.csv')        


#### L3 Testing stacked ML models on test_index #############################################

print('Calculating stacked ML on retest1 data ', datetime.now()) 

#apply trained stacked models ElasticNet to new data , test_index

#for presetet sets
dict_st_perf2={}
dict_st_ypred2={}
s=1
for set_n in [set1, set2, set3]:
    ftrs = df_y_pred3.loc[:, set_n]
    y_pred, y_real, ind_y, bacc, mse, corr, mae = reaply_ElNet(ftrs, y_res3, dict_st_models[('set'+str(s))]) #ML
    dict_st_ypred2[('set'+str(s))] = y_pred
    dict_st_perf2[('set'+str(s))] = bacc, mse, mae, corr
    s+=1

df_st_ypred2 = pd.DataFrame(dict_st_ypred2, index=ind_y)
df_st_perf2 = pd.DataFrame(dict_st_perf2, index=['best score r2', 'mse', 'mae','corr'])        

###Save outputs from this step (models and all mod. perf.)

#performance and prediction
df_st_perf2.to_csv(path_out+'/3level_retest1_perf_stacked.csv')
df_st_ypred2.to_csv(path_out+'/3level_retest1_y_pred_stacked.csv') 




### 3rd level ################################################################################
print(' ')
print('start 3rd level , retest2', datetime.now())


#### L3 Testing single ML models on retest2 #############################################

print('Checking single ML on retest2 data ', datetime.now())

#control for age+gen and age+gen+mov with sorting to index_retest_test

#control y (target) for age+gen
p1, p2 = control_2(y_retest_test2, control_retest_test2, index_retest_test)
y_res3 = pd.Series(p1, index = p2)

#control modalities
features_res3 = {}
for key in features_retest_test2.keys():

    #controlling tasks for 3 parameter (age+gen+mov)
    if key in ['emo', 'gam', 'lan', 'mot', 'rel', 'soc', 'wm', 'rest']:
        features_res3[key] = control_mov_feature(features_retest_test2[key], control_retest_test2, movements_retest_test2[key], index_retest_test)

    #controlling the remaining for 2 parameters (age+gen)
    if key in ['cort', 'surf', 'subc', 'VolBrain']:
        d = {}
        for col in features_retest_test2[key].columns:
            p1,p2 = control_2(features_retest_test2[key][col], control_retest_test2, index_retest_test)
            d[col] = p1
        df= pd.DataFrame(d, index = p2)
        features_res3[key] = df        


#apply trained single models ElasticNet to new data , test_index

dict_y_pred3={}
dict_y_pred3_per={}
for key in list(features_res3.keys()):
    y_pred, y_real, ind_y, bacc, mse, corr, mae = reaply_ElNet(features_res3[key], y_res3, dict_elnet_model[key]) #ML
    dict_y_pred3[key] = y_pred
    dict_y_pred3_per[key] = bacc, mse, mae, corr

df_y_pred3 = pd.DataFrame(dict_y_pred3, index=ind_y)
df_y_pred3_per = pd.DataFrame(dict_y_pred3_per, index=['best score r2', 'mse', 'mae','corr'])


###Save outputs from this step (models and all mod. perf.)

#model performance
df_y_pred3_per.to_csv(path_out+'/3level_retest2_perf_elnet.csv')

#list of first level targets (observed and predicted)
y_res3.to_csv(path_out+'/3level_retest2_y_real_residual_index.csv', header=None)
df_y_pred3.to_csv(path_out+'/3level_retest2_y_pred_singleML.csv')        


#### L3 Testing stacked ML models on test_index #############################################

print('Calculating stacked ML on retest2 data ', datetime.now()) 

#apply trained stacked models ElasticNet to new data , test_index

#for presetet sets
dict_st_perf2={}
dict_st_ypred2={}
s=1
for set_n in [set1, set2, set3]:
    ftrs = df_y_pred3.loc[:, set_n]
    y_pred, y_real, ind_y, bacc, mse, corr, mae = reaply_ElNet(ftrs, y_res3, dict_st_models[('set'+str(s))]) #ML
    dict_st_ypred2[('set'+str(s))] = y_pred
    dict_st_perf2[('set'+str(s))] = bacc, mse, mae, corr
    s+=1

df_st_ypred2 = pd.DataFrame(dict_st_ypred2, index=ind_y)
df_st_perf2 = pd.DataFrame(dict_st_perf2, index=['best score r2', 'mse', 'mae','corr'])        

###Save outputs from this step (models and all mod. perf.)

#performance and prediction
df_st_perf2.to_csv(path_out+'/3level_retest2_perf_stacked.csv')
df_st_ypred2.to_csv(path_out+'/3level_retest2_y_pred_stacked.csv')




print(' ')
print('finished to calculate')
print(datetime.now())




CogTotalComp_Unadj
 
started to calculate ML
2021-08-02 18:07:56.276605
 
start 1st level  2021-08-02 18:07:56.276721
start  emo 2021-08-02 18:09:33.430338
start  gam 2021-08-02 18:31:00.417822
start  lan 2021-08-02 19:03:18.109284
start  mot 2021-08-02 19:34:30.261354
start  rel 2021-08-02 21:46:44.014710
start  soc 2021-08-02 23:58:58.957912
start  wm 2021-08-03 00:20:18.265487
start  cort 2021-08-03 00:49:48.681936
start  subc 2021-08-03 01:05:13.422104
start  surf 2021-08-03 01:08:04.197299
start  rest 2021-08-03 01:21:33.449758
start  VolBrain 2021-08-03 07:38:09.811280
 
start 2nd level  2021-08-03 07:39:57.085022
Checking single ML on test1 data  2021-08-03 07:39:57.085076
Calculating stacked ML on test1 data  2021-08-03 07:41:00.274630
set 1 2021-08-03 07:41:00.275308
set 2 2021-08-03 07:43:20.885882
set 3 2021-08-03 07:45:18.035307
 
start 3rd level , retest1 2021-08-03 07:47:12.483973
Checking single ML on retest1 data  2021-08-03 07:47:12.484435
Calculating stacked ML on ret